In [1]:
import sys
sys.path.insert(0, '/home/vipul/llm/iterate')
print(sys.path)

['/home/vipul/llm/iterate', '/home/vipul/miniconda3/envs/llm/lib/python310.zip', '/home/vipul/miniconda3/envs/llm/lib/python3.10', '/home/vipul/miniconda3/envs/llm/lib/python3.10/lib-dynload', '', '/home/vipul/miniconda3/envs/llm/lib/python3.10/site-packages']


In [35]:
from dataset import JSONLDataset, TabularDataset, PickleDataset
import models.openai as openai
from util import parse_example, parse_tsv_example, parse_qaner_example, score_sets
import numpy as np
import time
import re

from dotenv import load_dotenv

import argparse
import random
from tqdm.auto import tqdm
import os, pdb
import json
import shutil
import logging
from datetime import datetime
import signal

In [42]:
def run_prompt(prompt, model):
    completion = model.complete(prompt)
    if completion is None or completion == '':
        return "Could not complete prompt"
    return completion

def construct_prompt(srcfile, promptfile, egs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}]', end=' ')
        print()

def get_prompt_eg_acc(srcfile, eg_idxs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}->{gtok}]', end=' ')
        print()
    
def create_model():
    load_dotenv('../../.env')
    openai.setup_api_key(os.environ.get('OPENAI_API_KEY'))

    model_args = openai.ChatGPT.DEFAULT_ARGS
    model_args['engine'] = 'gpt-4'
    model_args['request_timeout'] = 200
    return openai.ChatGPT(model_args)

def parse_response(response):
    lines = response.strip().split('\n')
    pred = [a.split('\t') for a in lines if '\t' in a]
    return list(zip(*pred))

input: I cha der ihri Telefonnummere gä , de nimmsch mou unverbindlech Kontakt uuf .

Examples:

59  
I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .  
PRON AUX [DET->ADP] PROPN VERB PUNCT PRON AUX PRON ADV ADJ CCONJ PRON [VERB->AUX] ADV [ADV->PART] VERB PUNCT   

4  
De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .  
DET NOUN AUX DET NOUN VERB PUNCT DET NOUN [VERB->AUX] DET NOUN VERB AUX PUNCT   

7  
Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .  
DET NOUN AUX ADV AUX PUNCT SCONJ PRON ADV ADV ADJ [ADP->AUX] AUX PUNCT SCONJ PRON ADV AUX AUX PART VERB PUNCT   

70  
Isch das e Sach gsi , bis mer se gfunge hei gha .  
AUX [DET->PRON] DET NOUN [VERB->AUX] PUNCT SCONJ PRON PRON VERB AUX [VERB->AUX] PUNCT   

61  
De Dialäkt muess zu de Gschecht und zum Inhaut vonere Werbig passe .  
DET NOUN AUX ADP DET NOUN CCONJ ADP NOUN ADP NOUN VERB PUNCT   

56  
Mit der Zit hani mi mit mir säuber uf ei Schriibwiis pro Wort aafo einige .  
ADP DET NOUN [PRON->AUX] PRON ADP PRON ADV ADP DET NOUN ADP NOUN VERB VERB PUNCT   

30  
Mit all denä Wörter hani natürli nüt chönä aafangä .  
ADP DET [PRON->DET] NOUN [VERB->AUX] ADV PRON [VERB->AUX] VERB PUNCT   

15  
Aso bini rächt uufgschmissä gsi und dem entschprächend fascht verzwiiflät .  
ADV AUX ADV [VERB->ADJ] AUX CCONJ PRON ADJ ADV VERB PUNCT   



In [20]:
eg_1_idxs = [66, 58, 93, 45, 15, 68, 26, 70]
eg_1_prompt = """Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .
Tags:
```
I	PRON
main	VERB
,	PUNCT
das	DET
Ganze	NOUN
letscht	ADJ
Wuchä	NOUN
isch	AUX
mier	PRON
scho	ADV
ächli	ADV
iigfaarä	VERB
.	PUNCT
```

Sentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .
Tags:
```
Du	PRON
gsehsch	VERB
uus	PRON
,	PUNCT
wi	SCONJ
wenn	SCONJ
de	DET
nöime	ADJ
no	ADV
hättisch	AUX
z	PART
trinken	VERB
übercho	VERB
.	PUNCT
```

Sentence: Dir weit mer doch nid verzöue , di Wäutsche heige vo eim Tag uf en anger ufghört Chuttlen ässe .
Tags:
```
Dir	PRON
weit	VERB
mer	PRON
doch	ADV
nid	ADV
verzöue	VERB
,	PUNCT
di	DET
Wäutsche	NOUN
heige	VERB
vo	ADP
eim	DET
Tag	NOUN
uf	ADP
en	DET
anger	ADJ
ufghört	VERB
Chuttlen	NOUN
ässe	VERB
.	PUNCT
```

Sentence: es isch nämli echt usgstorbe gsi .
Tags:
```
es	PRON
isch	AUX
nämli	ADV
echt	ADJ
usgstorbe	VERB
gsi	AUX
.	PUNCT
```

Sentence: Aso bini rächt uufgschmissä gsi und dem entschprächend fascht verzwiiflät .
Tags:
```
Aso	ADV
bini	AUX
rächt	ADV
uufgschmissä	VERB
gsi	AUX
und	CCONJ
dem	PRON
entschprächend	ADJ
fascht	ADV
verzwiiflät	VERB
.	PUNCT
```

Sentence: Der Ääschme wett nöd schaffe biin em .
Tags:
```
Der	DET
Ääschme	NOUN
wett	AUX
nöd	ADV
schaffe	VERB
biin	ADP
em	PRON
.	PUNCT
```

Sentence: Zerscht hends am Dani gsait , är söli dòch Hoochdütsch redä , das gängi denn grad gaar nöd , wenn är so redi , wiäner redi .
Tags:
```
Zerscht	ADV
hends	PRON
am	ADP
Dani	PROPN
gsait	VERB
,	PUNCT
är	PRON
söli	AUX
dòch	ADV
Hoochdütsch	ADJ
redä	VERB
,	PUNCT
das	PRON
gängi	VERB
denn	ADV
grad	ADV
gaar	ADV
nöd	ADV
,	PUNCT
wenn	SCONJ
är	PRON
so	ADV
redi	VERB
,	PUNCT
wiäner	PRON
redi	VERB
.	PUNCT
```

Sentence: Isch das e Sach gsi , bis mer se gfunge hei gha .
Tags:
```
Isch	AUX
das	PRON
e	DET
Sach	NOUN
gsi	AUX
,	PUNCT
bis	SCONJ
mer	PRON
se	PRON
gfunge	VERB
hei	AUX
gha	VERB
.	PUNCT
```

Sentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .
Tags:
```
"""

In [33]:
def find_and_replace_n_with_regex(text, target_word, replacement_word, n):
    pattern = r'\b{}\b'.format(target_word)
    count = 0
    def replacer(match):
        nonlocal count
        count += 1
        if count <= n:
            return replacement_word
        else:
            return match.group(0)

    return re.sub(pattern, replacer, text)

In [48]:
for i in range(1, 6):
    print(find_and_replace_n_with_regex("fish fish fish fish fish", "fish", "bait", i))

bait fish fish fish fish
bait bait fish fish fish
bait bait bait fish fish
bait bait bait bait fish
bait bait bait bait bait


In [31]:
model = create_model()

In [40]:
def print_results(prompt, model):
    tags = parse_response(run_prompt(prompt, model))
    print(' '.join(tags[0]))
    print(' '.join(tags[1]))

In [51]:
# 12 AUX in eg 1
for i in [6, 12]:
    print_results(find_and_replace_n_with_regex(eg_1_prompt, 'AUX', 'VERB', i), model)
    print()

Backing off 0.5 seconds after 1 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 0.4 seconds after 2 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 0.8 seconds after 3 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 3.1 seconds after 4 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 2.0 seconds after 5 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 15.9 seconds after 6 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , 

Backing off 61.7 seconds after 7 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , 

Backing off 115.5 seconds after 8 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPRON\n,\tPUNCT\nwi\tSCONJ\nwenn\tSCONJ\nde\tDET\nnöime\tADJ\nno\tADV\nhättisch\tAUX\nz\tPART\ntrinken\tVERB\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue ,

IndexError: list index out of range